In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(
  ["git", "clone", REPO_URL, str(REPO_DIR)],
  check=True
)

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {"--clean"}"
)

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : True

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [2]:
%%fenicsx

"""
FEniCSx Mass Matrix Assembly and Eigenvalue Problem Example

This example assembles mass and stiffness matrices on a 2D unit square domain
and computes eigenvalues of the Laplacian with Dirichlet boundary conditions.
Compatible with DOLFINx 0.10 - Using native PETSc backend
"""

import numpy as np
import ufl
from dolfinx import fem, mesh
import dolfinx.fem.petsc  # DOLFINx 0.10 requires this import style
from mpi4py import MPI
from petsc4py import PETSc

print("="*60)
print("FEniCSx Eigenvalue Problem: -Δu = λu")
print("Domain: Unit square [0,1] x [0,1]")
print("Boundary conditions: u = 0 on all boundaries")
print("="*60)

# 1. Create mesh (unit square)
domain = mesh.create_unit_square(
    MPI.COMM_WORLD,
    nx=32,  # Number of elements in x direction
    ny=32,  # Number of elements in y direction
    cell_type=mesh.CellType.triangle
)

# 2. Define function space (1st order Lagrange elements)
V = fem.functionspace(domain, ("Lagrange", 1))

print(f"\nNumber of DOFs: {V.dofmap.index_map.size_global}")

# 3. Define trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# 4. Define boundary: all edges of the unit square
def boundary(x):
    return np.logical_or(
        np.logical_or(np.isclose(x[0], 0), np.isclose(x[0], 1)),
        np.logical_or(np.isclose(x[1], 0), np.isclose(x[1], 1))
    )

# 5. Create Dirichlet boundary condition
boundary_dofs = fem.locate_dofs_geometrical(V, boundary)
bc = fem.dirichletbc(PETSc.ScalarType(0), boundary_dofs, V)

print(f"Number of boundary DOFs: {len(boundary_dofs)}")

# 6. Bilinear forms
# Mass matrix: M[i,j] = ∫_Ω φ_i·φ_j dx
mass_form = ufl.inner(u, v) * ufl.dx

# Stiffness matrix: K[i,j] = ∫_Ω ∇φ_i·∇φ_j dx
stiffness_form = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

# 7. Assemble matrices directly as PETSc matrices (native DOLFINx 0.10 way!)
print("\nAssembling matrices using native PETSc backend...")

# Assemble mass matrix with boundary conditions
M_petsc = dolfinx.fem.petsc.assemble_matrix(fem.form(mass_form), bcs=[bc])
M_petsc.assemble()

# Assemble stiffness matrix with boundary conditions
K_petsc = dolfinx.fem.petsc.assemble_matrix(fem.form(stiffness_form), bcs=[bc])
K_petsc.assemble()

print("Matrices assembled successfully (PETSc format)")
print(f"Mass matrix size: {M_petsc.getSize()}")
print(f"Stiffness matrix size: {K_petsc.getSize()}")

# Get matrix info
M_info = M_petsc.getInfo()
K_info = K_petsc.getInfo()
print(f"Mass matrix non-zeros: {int(M_info['nz_used'])}")
print(f"Stiffness matrix non-zeros: {int(K_info['nz_used'])}")

# 8. Note about boundary conditions
print("\n" + "="*60)
print("Note on Boundary Conditions:")
print("="*60)
print("With bcs=[bc] in assemble_matrix, boundary DOFs are handled")
print("by setting diagonal entries to 1 and off-diagonal to 0.")
print("This introduces spurious eigenvalues at λ=1.")
print("For clean eigenvalue spectrum, we need to remove these DOFs.")
print("="*60)

# 9. Extract interior DOFs submatrix
all_dofs = np.arange(V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts)
interior_dofs = np.setdiff1d(all_dofs, boundary_dofs)
interior_dofs = np.sort(interior_dofs).astype(np.int32)  # PETSc requires int32

print(f"\nExtracting interior DOF submatrices...")
print(f"Total DOFs: {len(all_dofs)}")
print(f"Boundary DOFs: {len(boundary_dofs)}")
print(f"Interior DOFs: {len(interior_dofs)}")

# Create index sets for submatrix extraction
is_interior = PETSc.IS().createGeneral(interior_dofs, comm=MPI.COMM_WORLD)

# Extract submatrices
M_interior = M_petsc.createSubMatrix(is_interior, is_interior)
K_interior = K_petsc.createSubMatrix(is_interior, is_interior)

print(f"\nReduced matrix size: {M_interior.getSize()}")
print(f"Interior mass matrix non-zeros: {int(M_interior.getInfo()['nz_used'])}")
print(f"Interior stiffness matrix non-zeros: {int(K_interior.getInfo()['nz_used'])}")

# 10. Solve generalized eigenvalue problem: K*u = λ*M*u
try:
    from slepc4py import SLEPc

    print("\n" + "="*60)
    print("Solving Eigenvalue Problem: K*u = λ*M*u")
    print("="*60)

    # Create eigenvalue solver
    eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD)
    eigensolver.setOperators(K_interior, M_interior)
    eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)  # Generalized Hermitian
    eigensolver.setDimensions(nev=10)  # Compute first 10 eigenvalues
    eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)
    eigensolver.setFromOptions()

    # Solve
    eigensolver.solve()

    # Get results
    nconv = eigensolver.getConverged()
    print(f"\nNumber of converged eigenvalues: {nconv}")
    print("\nEigenvalues (λ) with theoretical comparison:")
    print("-" * 70)
    print(f"{'Index':<8} {'Computed':<15} {'Theory':<15} {'(m,n)':<12} {'Error':<10}")
    print("-" * 70)

    # Theoretical eigenvalues for unit square: λ = π²(m² + n²)
    # Generate list with multiplicities expanded
    theoretical_list = []
    for m in range(1, 15):
        for n in range(1, 15):
            eigenval = np.pi**2 * (m**2 + n**2)
            theoretical_list.append((eigenval, m, n))

    # Sort by eigenvalue
    theoretical_list.sort(key=lambda x: x[0])

    # Print comparison
    for i in range(min(nconv, 10)):
        eigenvalue = eigensolver.getEigenvalue(i).real
        if i < len(theoretical_list):
            theory, m, n = theoretical_list[i]
            error = abs(eigenvalue - theory) / theory * 100
            print(f"{i+1:<8} {eigenvalue:<15.6f} {theory:<15.2f} ({m},{n}){'':<8} {error:<10.2f}%")
        else:
            print(f"{i+1:<8} {eigenvalue:<15.6f}")

    print("\n" + "="*60)
    print("Expected eigenvalues (with multiplicity expanded):")
    print("λ₁ = 2π²  ≈ 19.74   (1,1)")
    print("λ₂ = 5π²  ≈ 49.35   (1,2)")
    print("λ₃ = 5π²  ≈ 49.35   (2,1)")
    print("λ₄ = 8π²  ≈ 78.96   (2,2)")
    print("λ₅ = 10π² ≈ 98.70   (1,3)")
    print("λ₆ = 10π² ≈ 98.70   (3,1)")
    print("λ₇ = 13π² ≈ 128.30  (2,3)")
    print("λ₈ = 13π² ≈ 128.30  (3,2)")
    print("="*60)

except ImportError:
    print("\nSLEPc not installed. Install with: pip install slepc4py")

# 11. Test matrix-vector multiplication with PETSc
print("\n" + "="*60)
print("Matrix-Vector Multiplication Test:")
print("="*60)

# Create test vectors using PETSc
x = M_interior.createVecRight()
x.setRandom()
x.normalize()

# Compute y = M * x
y = M_interior.createVecLeft()
M_interior.mult(x, y)

# Compute z = K * x
z = K_interior.createVecLeft()
K_interior.mult(x, z)

print(f"Norm of vector x: {x.norm():.6f}")
print(f"Norm of vector y = M*x: {y.norm():.6f}")
print(f"Norm of vector z = K*x: {z.norm():.6f}")

# 12. Summary
print("\n" + "="*60)
print("Summary:")
print("="*60)
print(f"Mesh: {domain.topology.index_map(2).size_local} triangular elements")
print(f"Function space: P1 Lagrange elements")
print(f"Total DOFs: {V.dofmap.index_map.size_global}")
print(f"Boundary DOFs: {len(boundary_dofs)}")
print(f"Interior DOFs: {len(interior_dofs)}")
print(f"Matrix backend: PETSc (native DOLFINx 0.10)")
print(f"Assembly method: dolfinx.fem.petsc.assemble_matrix()")
print(f"External dependencies: NumPy, UFL, PETSc, SLEPc")

print("\nProgram completed successfully!")

FEniCSx Eigenvalue Problem: -Δu = λu
Domain: Unit square [0,1] x [0,1]
Boundary conditions: u = 0 on all boundaries

Number of DOFs: 1089
Number of boundary DOFs: 128

Assembling matrices using native PETSc backend...
Matrices assembled successfully (PETSc format)
Mass matrix size: (1089, 1089)
Stiffness matrix size: (1089, 1089)
Mass matrix non-zeros: 7361
Stiffness matrix non-zeros: 7361

Note on Boundary Conditions:
With bcs=[bc] in assemble_matrix, boundary DOFs are handled
by setting diagonal entries to 1 and off-diagonal to 0.
This introduces spurious eigenvalues at λ=1.
For clean eigenvalue spectrum, we need to remove these DOFs.

Extracting interior DOF submatrices...
Total DOFs: 1089
Boundary DOFs: 128
Interior DOFs: 961

Reduced matrix size: (961, 961)
Interior mass matrix non-zeros: 6481
Interior stiffness matrix non-zeros: 6481

Solving Eigenvalue Problem: K*u = λ*M*u

Number of converged eigenvalues: 11

Eigenvalues (λ) with theoretical comparison:
------------------------

In [3]:
%%fenicsx

"""
FEniCSx Eigenvalue Problem with Neumann Boundary Conditions

This example solves the eigenvalue problem:
    -Δu = λu  in Ω
    ∂u/∂n = 0  on ∂Ω  (Neumann BC)

Using real-valued PETSc for simplicity.
"""

import numpy as np
import ufl
from dolfinx import fem, mesh
import dolfinx.fem.petsc
from mpi4py import MPI
from petsc4py import PETSc

# Ensure PETSc uses real scalars
assert PETSc.ScalarType == np.float64 or PETSc.ScalarType == np.float32, \
    "This example requires real-valued PETSc"

print("="*60)
print("FEniCSx Eigenvalue Problem with Neumann BC")
print("Problem: -Δu = λu with ∂u/∂n = 0 on boundaries")
print("Domain: Unit square [0,1] x [0,1]")
print(f"PETSc scalar type: {PETSc.ScalarType}")
print("="*60)

# 1. Create mesh (unit square)
domain = mesh.create_unit_square(
    MPI.COMM_WORLD,
    nx=32,
    ny=32,
    cell_type=mesh.CellType.triangle
)

# 2. Define function space (1st order Lagrange elements)
V = fem.functionspace(domain, ("Lagrange", 1))

print(f"\nNumber of DOFs: {V.dofmap.index_map.size_global}")

# 3. Define trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# 4. Bilinear forms
# Mass matrix: M[i,j] = ∫_Ω φ_i·φ_j dx
mass_form = ufl.inner(u, v) * ufl.dx

# Stiffness matrix: K[i,j] = ∫_Ω ∇φ_i·∇φ_j dx
# Note: No boundary terms needed for Neumann BC ∂u/∂n = 0
stiffness_form = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

print("\n" + "="*60)
print("Key Difference: Neumann vs Dirichlet BC")
print("="*60)
print("Neumann (∂u/∂n = 0):")
print("  - Natural boundary condition")
print("  - Already satisfied by weak formulation")
print("  - NO boundary condition objects needed")
print("  - Use FULL matrix (no DOF removal)")
print("\nDirichlet (u = 0):")
print("  - Essential boundary condition")
print("  - Must be enforced explicitly")
print("  - Requires bc object and DOF removal")
print("="*60)

# 5. Assemble matrices WITHOUT any boundary conditions
print("\nAssembling matrices (no BC needed for Neumann)...")

M_petsc = dolfinx.fem.petsc.assemble_matrix(fem.form(mass_form))
M_petsc.assemble()

K_petsc = dolfinx.fem.petsc.assemble_matrix(fem.form(stiffness_form))
K_petsc.assemble()

print("Matrices assembled successfully")
print(f"Matrix size: {M_petsc.getSize()}")
print(f"Mass matrix non-zeros: {int(M_petsc.getInfo()['nz_used'])}")
print(f"Stiffness matrix non-zeros: {int(K_petsc.getInfo()['nz_used'])}")

# 6. Solve generalized eigenvalue problem: K*u = λ*M*u
try:
    from slepc4py import SLEPc

    print("\n" + "="*60)
    print("Solving Eigenvalue Problem: K*u = λ*M*u")
    print("="*60)

    # Create eigenvalue solver
    eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD)
    eigensolver.setOperators(K_petsc, M_petsc)
    eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    eigensolver.setDimensions(nev=15)  # Compute first 15 eigenvalues
    eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)
    eigensolver.setFromOptions()

    # Solve
    print("Computing eigenvalues...")
    eigensolver.solve()

    # Get results
    nconv = eigensolver.getConverged()
    print(f"\nNumber of converged eigenvalues: {nconv}")

    print("\n" + "="*60)
    print("Eigenvalues with Neumann BC:")
    print("="*60)
    print(f"{'Index':<8} {'Computed':<15} {'Theory':<15} {'(m,n)':<12} {'Error':<10}")
    print("-" * 70)

    # Theoretical eigenvalues for unit square with Neumann BC
    # λ = π²(m² + n²) where m,n = 0,1,2,...
    # Note: (0,0) gives λ = 0 (constant eigenfunction)
    theoretical_list = []
    for m in range(0, 15):
        for n in range(0, 15):
            eigenval = np.pi**2 * (m**2 + n**2)
            theoretical_list.append((eigenval, m, n))

    # Sort by eigenvalue
    theoretical_list.sort(key=lambda x: x[0])

    # Print comparison
    for i in range(min(nconv, 15)):
        eigenvalue = eigensolver.getEigenvalue(i).real
        if i < len(theoretical_list):
            theory, m, n = theoretical_list[i]
            if theory < 1e-10:
                # Zero eigenvalue
                print(f"{i+1:<8} {eigenvalue:<15.6e} {theory:<15.2f} ({m},{n}){'':<8} {'λ=0 (const)':<10}")
            else:
                error = abs(eigenvalue - theory) / theory * 100
                print(f"{i+1:<8} {eigenvalue:<15.6f} {theory:<15.2f} ({m},{n}){'':<8} {error:<10.2f}%")
        else:
            print(f"{i+1:<8} {eigenvalue:<15.6f}")

    print("\n" + "="*60)
    print("Expected eigenvalues for Neumann BC:")
    print("="*60)
    print("λ₀ = 0       (0,0) - constant function (trivial)")
    print("λ₁ = π²      (1,0) or (0,1) ≈ 9.87")
    print("λ₂ = π²      (0,1) or (1,0) ≈ 9.87 (degenerate)")
    print("λ₃ = 4π²     (2,0) or (0,2) ≈ 39.48")
    print("λ₄ = 4π²     (0,2) or (2,0) ≈ 39.48 (degenerate)")
    print("λ₅ = 2π²     (1,1) ≈ 19.74")
    print("λ₆ = 5π²     (1,2) or (2,1) ≈ 49.35")
    print("\nNote: Unlike Dirichlet BC, Neumann BC includes λ=0")
    print("="*60)

    # 7. Analyze eigenfunctions (now with real PETSc, much simpler!)
    print("\n" + "="*60)
    print("Eigenfunction Properties:")
    print("="*60)

    # Get first few eigenvectors
    for i in range(min(6, nconv)):
        eigenvalue = eigensolver.getEigenvalue(i)
        eigenvector = K_petsc.createVecRight()
        eigensolver.getEigenpair(i, eigenvector)

        # Normalize in L2 sense (using mass matrix)
        temp = M_petsc.createVecLeft()
        M_petsc.mult(eigenvector, temp)
        norm_M = np.sqrt(eigenvector.dot(temp))

        if norm_M > 1e-10:
            eigenvector.scale(1.0 / norm_M)

        # Compute integral ∫u dx = u^T * M * 1
        ones = M_petsc.createVecRight()
        ones.set(1.0)
        M_petsc.mult(ones, temp)  # temp = M * 1
        integral = eigenvector.dot(temp)

        # Compute L2 norm (should be 1 after normalization)
        M_petsc.mult(eigenvector, temp)
        norm_l2 = np.sqrt(eigenvector.dot(temp))

        # Classification
        if np.abs(eigenvalue) < 1e-6:
            classification = "Constant (rigid motion)"
        elif np.abs(integral) < 1e-3:
            classification = "Oscillatory (∫u≈0)"
        else:
            classification = f"Unexpected (∫u={integral:.3e})"

        print(f"λ_{i+1} = {eigenvalue:8.4f}: ∫u dx = {integral:10.6e}, ‖u‖_L2 = {norm_l2:.3f}  [{classification}]")

    print("\n" + "="*60)
    print("Theory: For Neumann BC on unit square")
    print("="*60)
    print("λ = 0:  Eigenfunction u = constant, ∫u dx ≠ 0  ✓")
    print("λ > 0:  Eigenfunctions are cos(mπx)cos(nπy)")
    print("        These satisfy ∫u dx = 0 (orthogonal to constant)")
    print("="*60)

except ImportError:
    print("\nSLEPc not installed. Install with: pip install slepc4py")

# 8. Test matrix-vector multiplication
print("\n" + "="*60)
print("Matrix-Vector Multiplication Test:")
print("="*60)

x = M_petsc.createVecRight()
x.setRandom()
x.normalize()

y = M_petsc.createVecLeft()
M_petsc.mult(x, y)

z = K_petsc.createVecLeft()
K_petsc.mult(x, z)

print(f"Norm of vector x: {x.norm():.6f}")
print(f"Norm of vector y = M*x: {y.norm():.6f}")
print(f"Norm of vector z = K*x: {z.norm():.6f}")

# 9. Summary
print("\n" + "="*60)
print("Summary:")
print("="*60)
print(f"Mesh: {domain.topology.index_map(2).size_local} triangular elements")
print(f"Function space: P1 Lagrange elements")
print(f"Total DOFs: {V.dofmap.index_map.size_global}")
print(f"Boundary condition: Neumann (∂u/∂n = 0)")
print(f"Matrix size: Full ({V.dofmap.index_map.size_global} × {V.dofmap.index_map.size_global})")
print(f"No DOF removal needed")

print("\n" + "="*60)
print("Key Differences from Dirichlet BC:")
print("="*60)
print("1. NO boundary condition object (bc) needed")
print("2. NO boundary DOF removal")
print("3. First eigenvalue λ₀ = 0 (constant function)")
print("4. Simpler implementation - just assemble and solve!")
print("="*60)

print("\nProgram completed successfully!")

FEniCSx Eigenvalue Problem with Neumann BC
Problem: -Δu = λu with ∂u/∂n = 0 on boundaries
Domain: Unit square [0,1] x [0,1]
PETSc scalar type: <class 'numpy.float64'>

Number of DOFs: 1089

Key Difference: Neumann vs Dirichlet BC
Neumann (∂u/∂n = 0):
  - Natural boundary condition
  - Already satisfied by weak formulation
  - NO boundary condition objects needed
  - Use FULL matrix (no DOF removal)

Dirichlet (u = 0):
  - Essential boundary condition
  - Must be enforced explicitly
  - Requires bc object and DOF removal

Assembling matrices (no BC needed for Neumann)...
Matrices assembled successfully
Matrix size: (1089, 1089)
Mass matrix non-zeros: 7361
Stiffness matrix non-zeros: 7361

Solving Eigenvalue Problem: K*u = λ*M*u
Computing eigenvalues...

Number of converged eigenvalues: 15

Eigenvalues with Neumann BC:
Index    Computed        Theory          (m,n)        Error     
----------------------------------------------------------------------
1        -4.033732e-13   0.00      